# Quant Finance - Clase 6 - Volatilidad Implicita

## Libreria Stock Options

Modulos necesarios

varios de library (en github),

matplotlib,

datetime,

sys

### Importo los modulos

In [ ]:
import sys
sys.path.append('..')
#Para poder leer los modulos dentro del directorio 
from library.stock import Stock
from library.options import StockOption
from library.market_data import MarketData
from library.curves import FlatZeroCurve
from library.volatility_surfaces import FlatVolatilitySurface
from library.bspricer import BlackScholesFormulaPricer
from library.mcpricer import SimpleMonteCarloPricer
from library.bsmodel import BlackScholesModel
from library.year_fractions import Actual360
import yfinance as yf
import datetime
from scipy.optimize import newton, brentq
from functools import partial
import matplotlib.pyplot as plt

#### 1) Fecha de valuación

In [ ]:
t = datetime.date(2020,8,12)

#### 2) Creamos el stock y agregamos solo su precio de cierre en t

In [ ]:
stock = Stock( name = 'AAPL' )
stock.add_close_price( time = t, price = 451.845 )

#### 3) Creamos la superficie de volatilidad y agregamos volatilidad al cierre de t

In [ ]:
vol_surface = FlatVolatilitySurface( name = 'AAPL_vol' )
vol_surface.add_close_volatility( time = t, strike = None, maturity = None, volatility = 0.2 )

#### 4) Creamos una curva de descuento flat y agregamos la tasa de descuento al cierre de t

In [ ]:
zero_curve = FlatZeroCurve( name = 'US_ZERO', year_fraction = Actual360 )
zero_curve.add_close_zero_rate( time = t, zero_rate = 0.0025 )

#### 5) Creamos el objeto que contiene la información del mercado

In [ ]:
market_data = MarketData()
market_data.add_stock( stock )
market_data.add_volatility_surface( vol_surface )
market_data.add_zero_curve( zero_curve )

#### 6) Definimos el instrumento de derivado (stock)

In [ ]:
maturity = datetime.date(2020,8,28)
option_type = 'call'
option = StockOption(   underlier_name = 'AAPL', 
                        strike = 235, 
                        maturity = maturity, 
                        payoff_name = option_type
                    )

#### 7) Calculamos el precio usando diferentes pricers

#### A) Formula de Black-Scholes

In [ ]:
pricer = BlackScholesFormulaPricer( option = option, market_data = market_data ) 
pricer_parameters = {'underlier_vol': 'AAPL_vol', 'zero_curve': 'US_ZERO'}
pricer.initialize_pricer(pricer_parameters)
print( 'BS formula:' )
print( pricer.npv(time = t) )

In [ ]:
strikes = [205. , 210. , 220. , 225. , 230. , 235. , 240. , 245. , 250. ,
       255. , 260. , 265. , 270. , 280. , 285. , 290. , 295. , 300. ,
       305. , 310. , 315. , 320. , 325. , 330. , 335. , 340. , 345. ,
       350. , 355. , 357.5, 360. , 362.5, 365. , 367.5, 370. , 372.5,
       375. , 377.5, 380. , 382.5, 385. , 387.5, 390. , 392.5, 395. ,
       397.5, 400. , 402.5, 405. , 407.5, 410. , 412.5, 415. , 420. ,
       425. , 430. , 435. , 440. , 445. , 450. , 455. , 460. , 465. ,
       470. , 475. , 480. , 485. , 490. , 495. , 500. , 505. , 510. ,
       520. , 530. , 540. , 550. , 560. , 570. , 580. , 590. , 600. ,
       610. , 620. , 630. , 640. , 650. , 660. , 670. , 680. , 690. ,
       700. ]

In [ ]:
prices = [2.4954e+02, 2.4595e+02, 2.3000e+02, 2.2000e+02, 2.2300e+02,
       2.1800e+02, 2.1250e+02, 2.0570e+02, 2.0530e+02, 1.9621e+02,
       1.9114e+02, 1.7182e+02, 1.8532e+02, 1.6572e+02, 1.7030e+02,
       1.6735e+02, 1.5800e+02, 1.5190e+02, 1.4523e+02, 1.4528e+02,
       1.2910e+02, 1.2200e+02, 1.2208e+02, 1.2770e+02, 1.1700e+02,
       1.1010e+02, 1.0817e+02, 1.0300e+02, 9.6400e+01, 8.2700e+01,
       9.1650e+01, 9.5250e+01, 8.6730e+01, 8.5900e+01, 8.0070e+01,
       8.0250e+01, 7.5300e+01, 7.5500e+01, 7.0700e+01, 6.4390e+01,
       6.8500e+01, 6.5000e+01, 6.3760e+01, 6.1000e+01, 5.8600e+01,
       5.5850e+01, 5.3440e+01, 5.0690e+01, 4.9570e+01, 4.6400e+01,
       4.4350e+01, 4.2350e+01, 3.9400e+01, 3.5300e+01, 3.1250e+01,
       2.6960e+01, 2.3350e+01, 2.0150e+01, 1.6850e+01, 1.4000e+01,
       1.1700e+01, 9.6300e+00, 7.9500e+00, 6.5800e+00, 5.5000e+00,
       4.5500e+00, 3.8500e+00, 3.2000e+00, 2.7500e+00, 2.3900e+00,
       2.0600e+00, 1.8200e+00, 1.4200e+00, 1.1500e+00, 9.4000e-01,
       7.9000e-01, 6.5000e-01, 5.5000e-01, 4.5000e-01, 4.0000e-01,
       3.3000e-01, 3.2000e-01, 3.1000e-01, 2.0000e-01, 1.6000e-01,
       1.7000e-01, 1.2000e-01, 5.0000e-02, 1.0000e-01, 7.0000e-02,
       8.0000e-02]

In [ ]:
strikes[40]

In [ ]:
prices[40]

In [ ]:
def c(vol, price, strike):
    vol_surface.add_close_volatility( time = t, strike = None, maturity = None, volatility = vol )
    option = StockOption(   underlier_name = 'AAPL', 
                            strike = strike, 
                            maturity = maturity, 
                            payoff_name = option_type
                        )
    pricer = BlackScholesFormulaPricer( option = option, market_data = market_data ) 
    pricer_parameters = {'underlier_vol': 'AAPL_vol', 'zero_curve': 'US_ZERO'}
    pricer.initialize_pricer(pricer_parameters)
    return price - pricer.npv(time = t)

In [ ]:
impl_vol = newton(c, 1, args = (prices[0], strikes[0]))
#impl_vol = brentq(c, 0.001, 2, args = (prices[0], strikes[0]))
impl_vol

In [ ]:
vol_surface.add_close_volatility( time = t, strike = None, maturity = None, volatility = impl_vol )
option = StockOption(   underlier_name = 'AAPL', 
                            strike = strikes[0], 
                            maturity = maturity, 
                            payoff_name = option_type
                        )
pricer = BlackScholesFormulaPricer( option = option, market_data = market_data ) 
pricer_parameters = {'underlier_vol': 'AAPL_vol', 'zero_curve': 'US_ZERO'}
pricer.initialize_pricer(pricer_parameters)
pricer.npv(time = t)

In [ ]:
impl_vols = []
for i in range(0,len(strikes)):
    try:
        impl_vol = newton(c, 1, args = (prices[i], strikes[i]))
        #impl_vol = brentq(c, 0.01, 2, args = (prices[i], strikes[i]))
    except:
        impl_vols.append(0)
    else:
        impl_vols.append(impl_vol)

In [ ]:
impl_vols

In [ ]:
plt.plot(strikes, impl_vols, 'ro')